In [1]:
import os
import argparse
import pickle
import joblib
import pdb

import pandas as pd
import numpy as np

from sklearn.model_selection import KFold



cohort_path = "/local-scratch/nigam/projects/jlemmon/cl-clmbr/experiments/main/data/"

In [2]:
def read_file(filename, columns=None, **kwargs):
    print(filename)
    load_extension = os.path.splitext(filename)[-1]
    if load_extension == ".parquet":
        return pd.read_parquet(filename, columns=columns,**kwargs)
    elif load_extension == ".csv":
        return pd.read_csv(filename, usecols=columns, **kwargs)

In [3]:
fn = cohort_path + "cohort/cohort_og.parquet"
fn2 = cohort_path + "cohort/cohort.parquet"

In [4]:
og_df = read_file(fn)
new_df = read_file(fn2)

/local-scratch/nigam/projects/jlemmon/cl-clmbr/experiments/main/data/cohort/cohort_og.parquet
/local-scratch/nigam/projects/jlemmon/cl-clmbr/experiments/main/data/cohort/cohort.parquet


In [12]:
og_df[['person_id', 'admit_date', 'discharge_date', 'admit_date_midnight',
       'discharge_date_midnight', 'hospital_mortality', 'death_date',
       'month_mortality', 'LOS_days', 'LOS_7', 'readmission_30',
       'readmission_window', 'icu_admission', 'icu_start_datetime',
       'age_in_years', 'age_group', 'race_eth', 'gender_concept_name',
       'race_eth_raw', 'race_eth_gender', 'race_eth_age_group',
       'race_eth_gender_age_group', 'race_eth_raw_gender',
       'race_eth_raw_age_group', 'race_eth_raw_gender_age_group',
       'prediction_id', 'fold_id']].head()

,person_id,admit_date,discharge_date,admit_date_midnight,discharge_date_midnight,hospital_mortality,death_date,month_mortality,LOS_days,LOS_7,...,gender_concept_name,race_eth_raw,race_eth_gender,race_eth_age_group,race_eth_gender_age_group,race_eth_raw_gender,race_eth_raw_age_group,race_eth_raw_gender_age_group,prediction_id,fold_id
0,29936887,2019-12-22 22:44:00,2019-12-26 16:00:00,2019-12-22 23:59:00,2019-12-26 23:59:00,0,NaT,0,4,0,...,FEMALE,Black or African American,race_eth:Black or African American-gender:FEMALE,race_eth:Black or African American-age_group:[...,race_eth:Black or African American-gender:FEMA...,race_eth_raw:Black or African American-gender:...,race_eth_raw:Black or African American-age_gro...,race_eth_raw:Black or African American-gender:...,-4386365341159031037,4
1,29936888,2010-07-12 06:34:00,2010-07-14 10:38:00,2010-07-12 23:59:00,2010-07-14 23:59:00,0,NaT,0,2,0,...,FEMALE,Black or African American,race_eth:Black or African American-gender:FEMALE,race_eth:Black or African American-age_group:[...,race_eth:Black or African American-gender:FEMA...,race_eth_raw:Black or African American-gender:...,race_eth_raw:Black or African American-age_gro...,race_eth_raw:Black or African American-gender:...,-5972979123358651111,1
2,29936900,2014-10-31 15:08:00,2014-11-03 13:25:00,2014-10-31 23:59:00,2014-11-03 23:59:00,0,NaT,0,3,0,...,FEMALE,Black or African American,race_eth:Black or African American-gender:FEMALE,race_eth:Black or African American-age_group:[...,race_eth:Black or African American-gender:FEMA...,race_eth_raw:Black or African American-gender:...,race_eth_raw:Black or African American-age_gro...,race_eth_raw:Black or African American-gender:...,-4691337650415059462,test
3,29936914,2018-06-23 05:20:00,2018-06-25 13:48:00,2018-06-23 23:59:00,2018-06-25 23:59:00,0,NaT,0,2,0,...,FEMALE,Black or African American,race_eth:Black or African American-gender:FEMALE,race_eth:Black or African American-age_group:[...,race_eth:Black or African American-gender:FEMA...,race_eth_raw:Black or African American-gender:...,race_eth_raw:Black or African American-age_gro...,race_eth_raw:Black or African American-gender:...,4763245573843019589,4
4,29936936,2018-02-08 12:05:00,2018-02-10 16:30:00,2018-02-08 23:59:00,2018-02-10 23:59:00,0,NaT,0,2,0,...,FEMALE,Black or African American,race_eth:Black or African American-gender:FEMALE,race_eth:Black or African American-age_group:[...,race_eth:Black or African American-gender:FEMA...,race_eth_raw:Black or African American-gender:...,race_eth_raw:Black or African American-age_gro...,race_eth_raw:Black or African American-gender:...,-5189004071480093272,2


In [16]:
new_df['prediction_id'] = og_df['prediction_id']

In [6]:
new_df.columns

Index(['person_id', 'admit_date', 'discharge_date', 'admit_date_midnight',
       'discharge_date_midnight', 'hospital_mortality', 'death_date',
       'month_mortality', 'LOS_days', 'LOS_7', 'readmission_30',
       'readmission_window', 'icu_admission', 'icu_start_datetime',
       'age_in_years', 'age_group', 'aki_base_creatinine',
       'aki_max_creatinine', 'aki1_creatinine', 'aki1_creatinine_time',
       'aki1_label', 'aki2_creatinine', 'aki2_creatinine_time', 'aki2_label',
       'hg_min_glucose', 'hg_glucose', 'hg_glucose_time', 'hg_label',
       'np_min_neutrophils', 'np_500_neutrophils', 'np_500_neutrophils_time',
       'np_500_label', 'np_1000_neutrophils', 'np_1000_neutrophils_time',
       'np_1000_label', 'race_eth', 'gender_concept_name', 'race_eth_raw',
       'race_eth_gender', 'race_eth_age_group', 'race_eth_gender_age_group',
       'race_eth_raw_gender', 'race_eth_raw_age_group',
       'race_eth_raw_gender_age_group', 'prediction_id', 'fold_id'],
      dtype='o